In [24]:
def generate_job_description(data):
    lines = []
    lines.append(f"{data['company_name']}에서 {data['position_role']} 포지션을 모집합니다.")

    if data.get('contract_type') and data.get('location'):
        lines.append(f"본 포지션은 {data['contract_type']}으로, {data['location']}에서 근무하게 됩니다.")
    elif data.get('contract_type'):
        lines.append(f"본 포지션은 {data['contract_type']}입니다.")
    elif data.get('location'):
        lines.append(f"근무지는 {data['location']}입니다.")

    if data.get('career'):
        lines.append(f"경력 조건은 {data['career']}입니다.")
    if data.get('education'):
        lines.append(f"학력 조건은 {data['education']}입니다.")
    if data.get('working_time'):
        lines.append(f"근무 시간은 {data['working_time']}입니다.")
    if data.get('skill'):
        lines.append(f"요구되는 기술은 {data['skill']}입니다.")
    if data.get('preferred'):
        lines.append(f"우대사항으로는 {data['preferred']}이 있습니다.")
    if data.get('salary'):
        lines.append(f"급여는 {data['salary']}입니다.")
    if data.get('due_date'):
        lines.append(f"접수 마감일은 {data['due_date']}입니다.")

    # 회사 소개
    company_info = []
    if data.get('founded'):
        company_info.append(f"{data['founded']}년에 설립")
    if data.get('employees'):
        company_info.append(f"현재 {data['employees']}의 직원")
    if data.get('company_type'):
        company_info.append(f"{data['company_type']}으로 분류")
    if data.get('industry'):
        company_info.append(f"산업 분야는 {data['industry']}")
    if company_info:
        lines.append(", ".join(company_info) + "입니다.")

    if data.get('homepage'):
        lines.append(f"자세한 정보는 회사 홈페이지({data['homepage']})에서 확인하실 수 있습니다.")

    return "\n".join(lines)


In [25]:
import os
import mysql.connector

rows = []

mysql_pw = os.environ.get('MYSQL')
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd=mysql_pw,
    database="job_matcher"
)

try:
    with conn.cursor(dictionary=True) as cursor:
        sql = """
        SELECT *
        FROM job_postings
        WHERE content = ""
        LIMIT 5;
        """
        cursor.execute(sql)
        while True:
            row = cursor.fetchone()
            if row is None:
                break
            rows.append(row)
finally:
    cursor.close()
    conn.close()


In [26]:
contents = []

for row in rows:
    contents.append(generate_job_description(row))

len(contents)

5

In [27]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.schema.runnable import RunnableLambda, RunnableSequence, RunnablePassthrough
api_key = os.environ.get("OPENAI_API_KEY")

model = ChatOpenAI(
    model = 'gpt-4.1',
    temperature = 0.1
)

response_schemas = [
    ResponseSchema(name="근무조건", description="간결한 근무조건 요약"),
    ResponseSchema(name="복리후생", description="간결한 복리후생 요약"),
    ResponseSchema(name="자격", description="간결한 자격요건 요약"),
    ResponseSchema(name="전형", description="간결한 전형절차 요약"),
    ResponseSchema(name="기타", description="기타 지원방법과 문의처 요약")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

results = []
for job_content in contents:
    prompt = ChatPromptTemplate.from_template(
        """
        너는 채용공고 요약 전문가야.
        다음 채용공고 본문을 읽고, 주어진 형식에 따라 JSON으로 간결하게 요약해줘.
        {format_instructions}

        채용공고 본문:
        \"\"\"
        {job_content}
        \"\"\"
        """
    )

    chain = (
        prompt
        | model
        | RunnableLambda(lambda x: output_parser.parse(x.content))
    )

    result = chain.invoke({
        "format_instructions" : format_instructions,
        "job_content" : job_content
    })
    results.append(result)
    print(result)
    print('*' * 20)

{'근무조건': '정규직(수습 3개월), 주5일(월~금) 08:00~17:00, 광주 광산구 근무, 급여는 회사 내규(면접 후 결정)', '복리후생': '내규에 따름(별도 언급 없음)', '자격': '신입·경력, 학력무관, Excel 활용 가능자, 유관업무 경험자 및 회계학과 우대', '전형': '서류전형 후 면접', '기타': '접수 마감 2025-09-14, 자세한 내용은 http://www.sjroad.kr 참고'}
********************
{'근무조건': '정규직, 경기도 부천시 근무, 주5일(월~금) 09:00~18:00, 월급 400만원, 경력·학력 무관', '복리후생': '중소기업 기준 복리후생 제공(상세 미기재)', '자격': '경력 및 학력 무관, 특별한 자격요건 없음', '전형': '서류 접수 후 개별 안내(전형 절차 미기재)', '기타': '접수 마감일 2025-08-15, 문의처 및 지원방법 미기재'}
********************
{'근무조건': '정규직(수습 3개월), 서울 강남구 근무, 주 5일 근무, 월 8회 휴무, 월급 236만원 이상', '복리후생': '대기업 수준 복리후생 제공(상세 내용 미기재)', '자격': '신입 및 경력, 고졸 이상, 인근 거주자 우대', '전형': '지원서 접수 후 전형(상세 절차 미기재), 접수 마감 2025-07-30', '기타': '회사 홈페이지(http://www.coffeebeankorea.com) 참고'}
********************
{'근무조건': '정규직, 계약직(정규직 전환 가능), 프리랜서(정규직 전환 가능) 중 선택, 경기도 고양시 일산동구 근무, 주 5일(월~금) 09:00~18:00 탄력근무제, 월급 500~850만원, 경력·학력 무관', '복리후생': '중소기업(비상장), 55명 규모, 12년차 기업', '자격': '경력·학력 무관, 운전 가능자 우대', '전형': '별도 전형절차 미기재', '기타': '지원방법 및 문의처 미기재'}
********

In [28]:
results[0]['근무조건']

'정규직(수습 3개월), 주5일(월~금) 08:00~17:00, 광주 광산구 근무, 급여는 회사 내규(면접 후 결정)'